# **Intro to Python for Data Analysis**
## Chapter 3: Data Manipulation
---
**Author:** Juan Martin Bellido  

**About**  
On this chapter we cover the two most typical data manipulation operations: *data aggregations* and *merging data tables*.

**Feedback?** Please share on [LinkedIn](https://www.linkedin.com/in/jmartinbellido/)  


## Table of Contents
---
1. Data aggregation
2. Merging data tables
3. Exercises


Conventions used in this document

> 👉 *This is note*

> ⚠️ *This is a warning*

# 1. Data aggregation
---

In the context of Data Analytics, to *aggregate* data means to summarize data from raw information.  

When aggregating data we always perform a calculation; we implement this by using an *aggregate function*.  

*List of aggregate functions*  

| Function    | Description                     |
|-------------|---------------------------------|
| count       | Number of non-null observations |
| nunique     | Number of unique values         |
| sum         | Sum of values                   |
| mean        | Mean of values                  |
| median      | Arithmetic median of values     |
| mode        | Mode                            |
| min         | Minimum                         |
| max         | Maximum     

In [ ]:
import pandas as pd

In [ ]:
# import df
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv")
df_jamesbond.dtypes

Film                  object
Year                   int64
Actor                 object
Director              object
Box Office           float64
Budget               float64
Bond Actor Salary    float64
dtype: object

### Basic aggregation

We begin performing simple aggregations by using aggregate functions on dataframes as methods.

```
df.agg_func()
```



In [ ]:
# aggregate function
df_jamesbond.max()
# we get max values for each of the fields available in df

In [ ]:
# we now filter one column before aggregating
df_jamesbond.Budget.sum().round()
# we nest a .round() method to round decimals

In [ ]:
# as a second example, we now apply a different agg func
df_jamesbond.Actor.nunique()
# note: there are 7 unique Actors in the dataset

### The `.agg()` method

Pandas includes a method that simplifies aggregating data.

```
df.agg({'column':'agg_func'})
```
> 👉 We use a dictionary to establish column and aggregate function

In [ ]:
# aggregating data using .agg() method
df_jamesbond.agg({'Actor':"nunique"})

In [ ]:
# aggregating column Box Office, three agg functions
df_jamesbond.agg({'Box Office':["min","max","mean"]})
# note that we are using three agg functions on the same column

In [ ]:
# we now aggregate more than one column
df_jamesbond.agg({
    'Budget':["min","max","median"],           
    'Bond Actor Salary':["min","max","median"]   
    })

### Grouped aggregations

When aggregating data, we might want to group observations in one or more categorical variables. We use the *groupby()* method to group rows in groups before aggregating data.

```
df.groupby("column")
```

In [ ]:
# we group by actor and aggregate two metrics
df_jamesbond.groupby("Actor")[["Bond Actor Salary","Box Office"]].max()

In [ ]:
# we repeat the exercise using the .agg() method
df_jamesbond.groupby("Actor").agg(
    {"Bond Actor Salary":"max","Box Office":"max"}
)

In [ ]:
# combining methods 
df_jamesbond.groupby("Actor")\
  .agg({"Bond Actor Salary":"max","Box Office":"mean"})\
  .sort_values("Bond Actor Salary", ascending=False)\
  .rename(columns={'Bond Actor Salary':'total_bond_salary','Box Office':'total_box_office'})


,total_bond_salary,total_box_office
Actor,,
Pierce Brosnan,17.9,471.650000
Daniel Craig,14.5,691.475000
Roger Moore,9.1,422.957143
Timothy Dalton,7.9,282.200000
Sean Connery,5.8,571.114286
George Lazenby,0.6,291.500000
David Niven,NaN,315.000000


In [ ]:
# aggregating grouped data
# we observe multi-index columns in case of performing multiple operations on the same field
df_jamesbond.groupby("Actor")\
  .agg({
      "Bond Actor Salary":["max","sum","mean","size"],
      "Budget":["max","min"]
    }
).sort_values(
    ("Bond Actor Salary","max") # we use a tuple to refer to twofold columns
    ,ascending=False
)


Bond Actor Salary                       Budget       
                             max   sum       mean size    max    min
Actor                                                               
Pierce Brosnan              17.9  46.5  11.625000    4  158.3   76.9
Daniel Craig                14.5  25.9   8.633333    4  206.3  145.3
Roger Moore                  9.1  16.9   8.450000    7   91.5   27.7
Timothy Dalton               7.9  13.1   6.550000    2   68.8   56.7
Sean Connery                 5.8  20.3   3.383333    7   86.0    7.0
George Lazenby               0.6   0.6   0.600000    1   37.3   37.3
David Niven                  NaN   0.0        NaN    1   85.0   85.0

# 2. Merging data tables
---



In [ ]:
# importing df
df_jamesbond = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/jamesbond.csv")
df_jamesbond.dtypes

Film                  object
Year                   int64
Actor                 object
Director              object
Box Office           float64
Budget               float64
Bond Actor Salary    float64
dtype: object

### Unions (merging rows)

We use the `pd.concat()` function to perform unions.

In [ ]:
pd.concat([df_jamesbond.head(2),df_jamesbond.tail(2)])

,Film,Year,Actor,Director,Box Office,Budget,Bond Actor Salary
0,Dr. No,1962,Sean Connery,Terence Young,448.8,7.0,0.6
1,From Russia with Love,1963,Sean Connery,Terence Young,543.8,12.6,1.6
24,Skyfall,2012,Daniel Craig,Sam Mendes,943.5,170.2,14.5
25,Spectre,2015,Daniel Craig,Sam Mendes,726.7,206.3,NaN


### Joins: merging tables by field in common

We use pandas `merge()` method to join two DataFrames. As in SQL, we can choose among different join types (i.e. *left, right, inner*).

```
df_1.merge(df_2,how='inner',on=None, ...)
```
In case fields in common are named differently on each table, we can specify names using the *left_on* and *right_on* paramaters.

```
df_1.merge(df_2,how='inner',left_on='key',right_on='key', ...)
```

In [ ]:
# importing df 1
df_albums = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/rolling_stones_top_metal_albums.csv')
df_albums.dtypes

AlbumID_Rank              int64
Artist                   object
Album                    object
Release_Year              int64
Spotify_Album            object
Description              object
wiki                     object
Duration                 object
Minutes                   int64
Seconds                   int64
Total_Seconds             int64
Label                    object
Sub_Metal_Genre          object
Rating                  float64
Rolling_Stone_Rating     object
dtype: object

In [ ]:
# importing df 2
df_songs = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/rolling_stones_top_metal_songs.csv')
df_songs.dtypes

Song Index       int64
Song            object
Artist          object
Track No.        int64
AlbumID_Rank     int64
BPM              int64
Energy           int64
Dance            int64
Loud             int64
Valence          int64
Duration        object
Acoustic         int64
Popularity       int64
Spotify_Song    object
dtype: object

In [ ]:
df_songs[['Song','Artist','AlbumID_Rank']].merge(
    df_albums[['AlbumID_Rank','Album','Release_Year']]  # df 2
    ,how='left'                                         # join type
    ,left_on='AlbumID_Rank'                             # key on left table 
    ,right_on='AlbumID_Rank'                            # key on right table
)

,Song,Artist,AlbumID_Rank,Album,Release_Year
0,War Pigs / Luke's Wall - 2014 Remaster,Black Sabbath,1,Paranoid,1970
1,Paranoid - 2016 Remaster,Black Sabbath,1,Paranoid,1970
2,Planet Caravan - 2013 Remaster,Black Sabbath,1,Paranoid,1970
3,Iron Man - 2014 Remaster,Black Sabbath,1,Paranoid,1970
4,Electric Funeral - 2013 Remaster,Black Sabbath,1,Paranoid,1970
...,...,...,...,...,...
1045,Sidewinder,Avenged Sevenfold,100,City of Evil,2005
1046,The Wicked End,Avenged Sevenfold,100,City of Evil,2005
1047,Strength of the World,Avenged Sevenfold,100,City of Evil,2005
1048,Betrayed,Avenged Sevenfold,100,City of Evil,2005


# 3. Exercises
---

> 👉 Solutions to exercises are available [here](https://nbviewer.org/github/SomosDataWizards/Python-Intro-Course/blob/main/Chapter_3_Exercises.ipynb)


### Exercise #1

A. Calculate total revenue by sector  
B. Repeat previous exercise but filtering for only companies in sector *Technology, Energy or Retailing*

> Dataset https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv



In [ ]:
import pandas as pd
df_fortune = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/fortune1000.csv")
df_fortune.dtypes

Rank          int64
Company      object
Sector       object
Industry     object
Location     object
Revenue       int64
Profits       int64
Employees     int64
dtype: object

### Exercise #2


Extract top 5 homeworlds with highest number of characters included in DataFrame.  

> Dataset https://data-wizards.s3.amazonaws.com/datasets/starwarsdb_people.csv





In [ ]:
import pandas as pd
df_starwars_people = pd.read_csv("https://data-wizards.s3.amazonaws.com/datasets/starwarsdb_people.csv")
df_starwars_people.dtypes

name           object
height        float64
mass          float64
hair_color     object
skin_color     object
eye_color      object
birth_year    float64
gender         object
homeworld      object
species        object
sex            object
dtype: object

### Exercise #3

Aggregate the following metrics by continent,

*   *Total population*
*   *Average GDP per capita*
*   *Average % of population living below poberty line*

> Dataset https://data-wizards.s3.amazonaws.com/datasets/dataset_na_who.csv




In [ ]:
import pandas as pd
df_who = pd.read_csv('https://data-wizards.s3.amazonaws.com/datasets/dataset_na_who.csv')
df_who.dtypes

Country                                                    object
CountryID                                                   int64
ContinentID                                                 int64
Adolescent fertility rate (%)                             float64
Adult literacy rate (%)                                   float64
Gross national income per capita (PPP international $)    float64
Net primary school enrolment ratio female (%)             float64
Net primary school enrolment ratio male (%)               float64
Population (in thousands) total                           float64
Population annual growth rate (%)                         float64
Population in urban areas (%)                             float64
Population living below the poverty line                  float64
Continent                                                  object
dtype: object